# DATA 604 Final Project: An Assessment of the Commercial Aviation Industry
#### Amanda Chan, Ryan Kasserra, Manas Shukla, David St. George 

## Introduction
*(Why did you choose this project and why is it important? You can include any background research you felt was important, and any research questions you would like to discuss in the remainder of the report)*

As the world attempts to settle into a post-pandemic state of normalcy in 2021, the air travel industry is beginning its recovery. In the United States, smaller regional airports have spearheaded this recovery – with some seeing 12% greater traffic than the same time last year (as of April). Most of this rebound can be attributed to travelers seeking outdoor vacation destinations and those from less risk-averse locations (Bui & Kliff, 2021). A return to pre-pandemic travel behavior by the public is projected to be soon and is only held back by a lagging recovery on the part of business travel. Knowing this, we attempt to revisit domestic U.S. air travel data in order to find patterns which offer insight on pricing, and delays. With the ongoing climate crisis – we also explore questions regarding the viability of more carbon-efficient alternatives to air travel such as high-speed rail. Finally, we inspect the opinions that travelers hold towards their airline and air-travel experience.



In [1]:
# imports needed to run the notebook
import csv
import mysql.connector
import datetime
import time
import pandas as pd
import sqlalchemy as sq

In [3]:
# (sql server connection details redacted)

cursor = connection.cursor()

# Execute a 'Show Tables' statement to verify that your relational table has been created.
cursor.execute("SHOW TABLES;") 

# get results of query
display(cursor.fetchall())

[('air_carrier_statistics',),
 ('average_fare_annual_2019',),
 ('average_fare_q1_2019',),
 ('average_fare_q2_2019',),
 ('average_fare_q3_2019',),
 ('average_fare_q4_2019',),
 ('otp',),
 ('otp2',),
 ('perairline',),
 ('tweets',)]

***
## Individual Datasets
*(What was each individual dataset, why did you choose it, and what did you learn from it? Include in this section a discussion of any steps you had to take to clean or extract the data you were working on. Ideally, this will be a summary/compilation of work you have already done, for the individual data exploration)*

***
### Dataset #3: T-100 Domestic Market All Carriers (Passenger Traffic): 
__Responsible team member: David St. George__

__Format:__ This dataset was found on the US Department of Transportation's Bureau of Transportation Statistics website and is in the public domain. The dataset consists of 1 file in csv format, is 61.8 MB, and contains 36 columns. The dataset contains aggregated flight information by month *and* by route and includes: the number of passengers on-board, freight carried (pounds), mail carried (pounds), route distance (in 500 mile categories), the carrier, the origin and destination, month, and the service class for 2019. Of particular interest are the columns identifying the distance flown (either raw distance or categorized), the time period of the flight (month), and the type of cargo. 

__Cleaning & Loading:__ The dataset was quite clean and did not require much maintenance prior to importing onto the MariaDB server. All of the columns included in the raw dataset have been imported although the majority are not used in the data exploration process. This was done to provide options for merging with the other datasets in the project. The lone cleaning exercise completed was to remove all flights/routes which were not designated as Service Class 'F', meaning Scheduled Passenger/Cargo Service. This filtered out Private or Charter planes, which were not intended as part of the analysis. 

__Challenges:__ A slight challenge was gaining a complete understanding of each column in the dataset since the keys in a couple of the columns are not intuitive. Some time was spent during the initial exploration stages filtering out desired information in a much more cumbersome way which would have been prevented by reading through all of the key tables before starting. 



In [2]:
# Create Air Carrier Statistics (ACS) Table

def create_acs_table():
    
    all_carrier = pd.read_csv("2019_T100_ALL_CARRIER.csv")
    all_carrier = all_carrier.iloc[:, :-1]
    
# (connection details redacted)

    create_table_statement = ('''CREATE TABLE IF NOT EXISTS `air_carrier_statistics` (
        `PASSENGERS` INT(32),
        `FREIGHT` INT(32),
        `MAIL` INT(32),
        `DISTANCE` INT(16) NOT NULL, 
        `UNIQUE_CARRIER` VARCHAR(4) NOT NULL,
        `AIRLINE_ID` VARCHAR(5) NOT NULL,
        `UNIQUE_CARRIER_NAME` VARCHAR(100) NOT NULL, 
        `UNIQUE_CARRIER_ENTITY` VARCHAR(4) NOT NULL,
        `REGION` VARCHAR(1) NOT NULL, 
        `CARRIER` VARCHAR(4) NOT NULL,
        `CARRIER_NAME` VARCHAR(100) NOT NULL,
        `CARRIER_GROUP` VARCHAR(1) NOT NULL,
        `CARRIER_GROUP_NEW` VARCHAR(1) NOT NULL,
        `ORIGIN_AIRPORT_ID` VARCHAR(5) NOT NULL,
        `ORIGIN_AIRPORT_SEQ_ID` VARCHAR(7) NOT NULL,
        `ORIGIN_CITY_MARKET_ID` VARCHAR(5) NOT NULL,
        `ORIGIN` VARCHAR(3) NOT NULL,
        `ORIGIN_CITY_NAME` VARCHAR(40) NOT NULL,
        `ORIGIN_STATE_ABR` VARCHAR(2) NOT NULL,
        `ORIGIN_STATE_FIPS` VARCHAR(2) NOT NULL,
        `ORIGIN_STATE_NM` VARCHAR(40) NOT NULL,
        `ORIGIN_WAC` VARCHAR(2) NOT NULL, 
        `DEST_AIRPORT_ID` VARCHAR(5) NOT NULL,
        `DEST_AIRPORT_SEQ_ID` VARCHAR(7) NOT NULL,
        `DEST_CITY_MARKET_ID` VARCHAR(5) NOT NULL,
        `DEST` VARCHAR(3) NOT NULL,
        `DEST_CITY_NAME` VARCHAR(40) NOT NULL,
        `DEST_STATE_ABR` VARCHAR(2) NOT NULL,
        `DEST_STATE_FIPS` VARCHAR(2) NOT NULL,
        `DEST_STATE_NM` VARCHAR(40) NOT NULL,
        `DEST_WAC` VARCHAR(2) NOT NULL,
        `YEAR` INT(16) NOT NULL,
        `QUARTER` INT(8) NOT NULL,
        `MONTH` INT(8) NOT NULL,
        `DISTANCE_GROUP` INT(8) NOT NULL,
        `CLASS` VARCHAR(1) NOT NULL
        );
        ''')
    
    create_cursor = cnx.cursor()
    create_cursor.execute(create_table_statement)
    create_cursor.close()
    
    # Connect to Group Schema
    # (connection details redacted)
    
    # Import Data into SQL Table
    all_carrier.to_sql(name='air_carrier_statistics', con=engine, schema='L03-9', chunksize=250, method='multi', if_exists='replace')

    # Drop Private Flights From Dataset
    clean_table_statement = (
        '''
        DELETE FROM `air_carrier_statistics` 
        WHERE `CLASS` <> 'F';
        ''')

    cursor = cnx.cursor()
    cursor.execute(clean_table_statement)
    cnx.commit()
    
       # Change Dallas Name Title to Include Dallas/Fort Worth
    update_table_statement = (
        '''
        UPDATE `air_carrier_statistics`
        SET ORIGIN_CITY_NAME = 'Dallas/Fort Worth, TX'
        WHERE ORIGIN = 'DAL';
        
        UPDATE `air_carrier_statistics`
        SET DEST_CITY_NAME = 'Dallas/Fort Worth, TX'
        WHERE DEST = 'DAL';
        ''')
    cursor.execute(update_table_statement)    
    
    cursor.close()
    cnx.close()

#create_acs_table()

In [86]:
# show what 5 rows of the dataset looks like
passenger_demo = pd.read_sql_query('''SELECT * FROM air_carrier_statistics LIMIT 3;''', engine)
display(passenger_demo)

,index,PASSENGERS,FREIGHT,MAIL,DISTANCE,UNIQUE_CARRIER,AIRLINE_ID,UNIQUE_CARRIER_NAME,UNIQUE_CARRIER_ENTITY,REGION,...,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS
0,960,0.0,0.0,0.0,180.0,9X,21618,"Southern Airways Express, dba Mokulele Airlines",01168,D,...,"Key West, FL",FL,12,Florida,33,2019,4,10,1,F
1,1167,0.0,0.0,0.0,183.0,9K,20253,Cape Air,07021,D,...,"Hyannis, MA",MA,25,Massachusetts,13,2019,4,10,1,F
2,1168,0.0,0.0,0.0,194.0,9K,20253,Cape Air,07021,D,...,"Provincetown, MA",MA,25,Massachusetts,13,2019,4,10,1,F


__The dataset was chosen to explore generally the following question:__ <u>What insight does airport traffic data offer?</u> How much does the time of year impact the number of people flying? What percentage of these flights & passengers are short-haul (< 500 miles) verses long-haul? Is there the potential that short-haul travel may be impacted by alternative modes of transportation like high-speed rail? 
***
__The following queries were performed to explore this dataset further:__

**Query 1**

In [6]:
# Query #1 
# Purpose: How does the number of passengers change based on month or quarter? 

# connection details redacted


query_table = pd.read_sql_query(
    '''
    SELECT QUARTER as `Quarter`, SUM(Passengers) as `Total Passengers`, round(SUM(Passengers)*100/(SELECT SUM(Passengers) FROM air_carrier_statistics),2) as `% of Total`
    FROM air_carrier_statistics
    GROUP BY Quarter
    ORDER BY SUM(Passengers) DESC;
    ''', engine)
display(query_table)

query_table = pd.read_sql_query(
    '''
    SELECT MONTH as `Month`, SUM(Passengers) as `Total Passengers`, round(SUM(Passengers)*100/(SELECT SUM(Passengers) FROM air_carrier_statistics),2) as `% of Total`
    FROM air_carrier_statistics
    GROUP BY MONTH
    ORDER BY SUM(Passengers) DESC;
    ''', engine)

display(query_table)
engine.dispose()

,Quarter,Total Passengers,% of Total
0,3,212002480.0,26.12
1,2,211093217.0,26.01
2,4,204501691.0,25.20
3,1,183947872.0,22.67


,Month,Total Passengers,% of Total
0,7,75281916.0,9.28
1,6,72790418.0,8.97
2,8,72729199.0,8.96
3,5,71364145.0,8.79
4,3,70234129.0,8.65
5,10,69936836.0,8.62
6,12,69737438.0,8.59
7,4,66938654.0,8.25
8,11,64827417.0,7.99
9,9,63991365.0,7.89


From the queries, we can see that Q3 is narrowly the busiest quarter of the year and Q1 the slowest. The gap between Spring and Summer are not as large as expected.
Analyzing by month, the months of June, July, and August have the most passengers and September, January, and February the least. 

This query is important as it provides general knowledge about the dataset and aviation traffic patterns based on time.

**Query 2**

In [7]:
# Query #2
# Short Haul Flights are those classed as < 500 miles (or Distance_Group '1')
# Purpose: To gain an understanding of what % of all Domestic Passengers are Short-Haul 

query_table = pd.read_sql_query(    
    '''
    SELECT round(SUM(Passengers)*100 / (SELECT SUM(Passengers) FROM air_carrier_statistics),2) as `Short Haul Passenger %`, 
    SUM(Passengers) as `Total Short Haul Passengers`
    FROM air_carrier_statistics
    WHERE DISTANCE_GROUP = '1';
    ''', engine)

display(query_table)

# Table 2
query_table = pd.read_sql_query(    
    '''
    SELECT A.ORIGIN_STATE_NM as `State`, A.SH_PASS as `Total Short Haul Passengers`, 100*ROUND(A.SH_PASS/B.TOT_PASS,2) as `% of All State-Originated Traffic`
    FROM 
    (
    SELECT ORIGIN_STATE_NM, SUM(Passengers) as `SH_PASS`
    FROM air_carrier_statistics
    WHERE DISTANCE_GROUP = '1'
    GROUP BY ORIGIN_STATE_NM
    ) A
    INNER JOIN
    (
    SELECT ORIGIN_STATE_NM, Sum(Passengers) as `TOT_PASS`
    FROM air_carrier_statistics
    GROUP BY ORIGIN_STATE_NM
    ) B
    ON A.ORIGIN_STATE_NM = B.ORIGIN_STATE_NM
    ORDER BY 2 DESC
    LIMIT 10;
    ''', engine)
display(query_table)


,Short Haul Passenger %,Total Short Haul Passengers
0,29.88,242454563.0


,State,Total Short Haul Passengers,% of All State-Originated Traffic
0,California,34356841.0,35.0
1,Texas,20843559.0,26.0
2,Georgia,18252171.0,37.0
3,North Carolina,15208478.0,47.0
4,Illinois,11245904.0,25.0
5,Florida,11191559.0,15.0
6,Virginia,9852233.0,40.0
7,Nevada,9004205.0,36.0
8,New York,8373362.0,22.0
9,Michigan,7345740.0,38.0


From the queries above, we can see that passengers on flights of 500 miles or less account for close to 30% of all passengers.\
Looking state-by-state, California and Texas have the most amount of short-haul passengers. This is not a huge surprise given they are the two most populated states in the country, but the presence of Georgia and North Carolina immediately behind is interesting although they are the 8th and 9th most populated states respectively. This is especially true when comparing them with Florida and New York, who lie behind them in this table but have double the population.

This is important as it provides insight into the geographical trends with respect to this particular mode of travel in the United States.

**Query 3**

In [39]:
# Query #3-a)
# Purpose: Understanding where the most Short-Haul passengers originate could provide 
# a starting point for considering a High Speed Rail Network in the United States

print('Top 10 Cities for Short-Haul Passengers')

query_table = pd.read_sql_query(    
    '''
    SELECT A.`City Name`, A.`Short-Haul Passengers`, A.`% of All Short-Haul Passengers Nationwide`, B.`TOT_RANK` as `Rank Overall # Passengers`
    FROM 
    (   
    SELECT ORIGIN_CITY_NAME as `City Name`, SUM(Passengers) as `Short-Haul Passengers`,
    ROUND(SUM(Passengers)*100/(SELECT SUM(Passengers) FROM air_carrier_statistics WHERE DISTANCE_GROUP = '1'),2) as `% of All Short-Haul Passengers Nationwide`
    FROM air_carrier_statistics
    WHERE DISTANCE_GROUP = '1'  
    GROUP BY ORIGIN_CITY_NAME
    ORDER BY 3 DESC
    LIMIT 10
    ) A
    INNER JOIN
    (
    SELECT ORIGIN_CITY_NAME, SUM(Passengers) as `TOT_PASS`, RANK() OVER(ORDER BY SUM(PASSENGERS) DESC) as `TOT_RANK`
    FROM air_carrier_statistics
    GROUP BY ORIGIN_CITY_NAME        
    ) B
    ON A.`City Name` = B.ORIGIN_CITY_NAME
    ORDER BY 2 DESC
    ;
    ''', engine)

display(query_table)


# Query 3-b)
# Purpose: Taking the Top 25 Airports for Overall Passenger Traffic, 
# Find Those That Have the Highest % of Domestic Travellers

print("Top 25 Cities for Overall Passenger Traffic Ranked By % of Short-Haul Passengers")
query_table2 = pd.read_sql_query(    
    '''
    SELECT SHORT_HAUL_PASSENGERS.ORIGIN_CITY_NAME as `City Name`, 
    100*ROUND(SHORT_HAUL_PASSENGERS.SH_PASS/TOTAL_PASSENGERS.TOT_PASS,2) as `% Short-Haul Passengers`, 
    TOTAL_PASSENGERS.TOT_RANK as `Rank Overall # Passengers`
    FROM
    (
        SELECT ORIGIN_CITY_NAME, SUM(Passengers) as `TOT_PASS`, RANK() OVER(ORDER BY SUM(PASSENGERS) DESC) as `TOT_RANK`
        FROM air_carrier_statistics
        GROUP BY ORIGIN_CITY_NAME        
    ) TOTAL_PASSENGERS
    INNER JOIN
    (
        SELECT ORIGIN_CITY_NAME, SUM(Passengers) as `SH_PASS` 
        FROM air_carrier_statistics
        WHERE DISTANCE_GROUP = '1'
        GROUP BY ORIGIN_CITY_NAME
    ) SHORT_HAUL_PASSENGERS
    ON TOTAL_PASSENGERS.ORIGIN_CITY_NAME = SHORT_HAUL_PASSENGERS.ORIGIN_CITY_NAME
    WHERE TOTAL_PASSENGERS.TOT_RANK < 25
    ORDER BY 2 DESC
    LIMIT 10;
    ''', engine)
    
display(query_table2)



Top 10 Cities for Short-Haul Passengers


,City Name,Short-Haul Passengers,% of All Short-Haul Passengers Nationwide,Rank Overall # Passengers
0,"Atlanta, GA",17139860.0,7.07,1
1,"Chicago, IL",10837304.0,4.47,2
2,"Charlotte, NC",9903133.0,4.08,9
3,"Las Vegas, NV",7722780.0,3.19,8
4,"Los Angeles, CA",7105181.0,2.93,5
5,"Washington, DC",6701991.0,2.76,14
6,"Dallas/Fort Worth, TX",6356580.0,2.62,4
7,"Detroit, MI",5655758.0,2.33,17
8,"Houston, TX",5540074.0,2.28,7
9,"San Francisco, CA",5365183.0,2.21,13


Top 25 Cities for Overall Passenger Traffic Ranked By % of Short-Haul Passengers


,City Name,% Short-Haul Passengers,Rank Overall # Passengers
0,"Charlotte, NC",44.0,9
1,"San Diego, CA",37.0,23
2,"Atlanta, GA",36.0,1
3,"Detroit, MI",35.0,17
4,"Washington, DC",35.0,14
5,"Baltimore, MD",34.0,21
6,"Las Vegas, NV",34.0,8
7,"Philadelphia, PA",30.0,19
8,"Boston, MA",27.0,16
9,"San Francisco, CA",26.0,13


The first query provides an overall summary indicating the Top 10 total number of short-haul passengers by city nationwide. As can be seen in the percentages in the third column, the percentage of overall short-haul traffic by city is not high indicating the spread of short-haul traffic over the entire country. The final column with the Overall Rank was included to provide context of how busy these cities are with passengers in general. 

The second query looks at locations with high numbers of overall traffic where short-haul flights are most prevalent to try and identify regions where short-haul traffic is extremely relied upon as part of the transportation infrastructure.

Given the previous results, it is not a great surprise to see Atlanta, GA and Charlotte, NC top each list. This would lead to the inference that they are hubs serving a much larger region of multiple states than simply their respective city or state. This makes some sense given the population growth in the state of North Carolina over the past ten years (Tippett, 2019).

**Query 4**

In [40]:
# Query 4 
# Purpose: 

query_table2 = pd.read_sql_query(    
    '''
    SELECT TOT_PASS.ORIGIN_CITY_NAME `City Name`, 
    TOT_PASS.TOT_RANK `Domestic Passenger Rank`, ROUND(FLGHT_MEAN.WeightedMean, 2) AS `Weighted Mean Flight Distance (mi)`
    FROM
    (
    SELECT ORIGIN, ORIGIN_CITY_NAME, SUM(Passengers) as `TOT_PASS`, RANK() OVER(ORDER BY SUM(PASSENGERS) DESC) as `TOT_RANK`
    FROM air_carrier_statistics
    GROUP BY ORIGIN_CITY_NAME        
    ) TOT_PASS
    INNER JOIN 
    (
    SELECT ORIGIN, ORIGIN_CITY_NAME, (SUM(PASSENGERS*DISTANCE)/SUM(PASSENGERS)) as `WeightedMean`
    FROM air_carrier_statistics
    WHERE PASSENGERS <> 0
    GROUP BY ORIGIN_CITY_NAME
    ) FLGHT_MEAN
    ON TOT_PASS.ORIGIN_CITY_NAME = FLGHT_MEAN.ORIGIN_CITY_NAME
    WHERE TOT_PASS.TOT_RANK < 25
    ORDER BY 3 ASC
    LIMIT 10;
    ''', engine)

print("Top 25 Cities for Overall Passenger Traffic Ranked By Weight Mean Flight Distance")
display(query_table2)

# Table 2 - Looking at Charlotte Individually 
print('Individual View of Charlotte')
query_table = pd.read_sql_query(    
    '''
    SELECT ORIGIN, ORIGIN_CITY_NAME, DEST, DEST_CITY_NAME, SUM(Passengers) As `Total Passengers`, DISTANCE as `Distance (mi)`
    FROM air_carrier_statistics
    WHERE DISTANCE_GROUP = '1' AND
    ORIGIN_CITY_NAME = 'Charlotte, NC'
    GROUP BY ORIGIN, DEST
    ORDER BY SUM(Passengers) DESC
    LIMIT 5;
    ''', engine)
display(query_table)


Top 25 Cities for Overall Passenger Traffic Ranked By Weight Mean Flight Distance


,City Name,Domestic Passenger Rank,Weighted Mean Flight Distance (mi)
0,"Charlotte, NC",9,652.00
1,"Atlanta, GA",1,726.28
2,"Detroit, MI",17,812.43
3,"Baltimore, MD",21,870.31
4,"Washington, DC",14,878.00
5,"Chicago, IL",2,881.91
6,"Salt Lake City, UT",22,895.27
7,"Houston, TX",7,908.13
8,"Dallas/Fort Worth, TX",4,909.20
9,"Minneapolis, MN",15,936.68


Individual View of Charlotte


,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,Total Passengers,Distance (mi)
0,CLT,"Charlotte, NC",MCO,"Orlando, FL",674035.0,468.0
1,CLT,"Charlotte, NC",ATL,"Atlanta, GA",542866.0,226.0
2,CLT,"Charlotte, NC",BWI,"Baltimore, MD",512252.0,361.0
3,CLT,"Charlotte, NC",PHL,"Philadelphia, PA",500427.0,449.0
4,CLT,"Charlotte, NC",RDU,"Raleigh/Durham, NC",369386.0,130.0


This query examines a similar question as the previous - attempting to indentify cities with a high volume of traffic relative nationally that are serving passengers for primarily short-haul traffic. The metric of 'Weighted Mean Flight Distance' takes the summation of the number of passengers multipled by the distance for each route and divides by the number of passengers. This provides another way of looking at amount of traffic by distance to see where the largest 'regional hubs' are located. 

I also wanted to perform an additional query on Charlotte itself, as it is a curious case to see where most of its traffic comes from. It looks as if it is very much a mid-Atlantic hub connecting the North and South.

**Query 5**

In [87]:
# Query 5
# Purpose: What are the most popular short-haul routes? Do they align with the results found above? 

query_table = pd.read_sql_query(    
    '''
    SELECT ORIGIN, ORIGIN_CITY_NAME, DEST, DEST_CITY_NAME, SUM(Passengers) As `Total Passengers`
    FROM air_carrier_statistics
    WHERE DISTANCE_GROUP = '1'
    GROUP BY ORIGIN, DEST
    ORDER BY SUM(Passengers) DESC
    LIMIT 10;
    ''', engine)
display(query_table)

# Table 2: Create a Passenger/Distance Ratio to examine number of passengers per mile to examine popular short-haul routes 
query_table2 = pd.read_sql_query(    
    '''
    SELECT ORIGIN, ORIGIN_CITY_NAME, DEST, DEST_CITY_NAME, ROUND((SUM(PASSENGERS)/DISTANCE),2) AS `Passenger:Distance Ratio`, DISTANCE as `Distance`
    FROM air_carrier_statistics
    WHERE PASSENGERS <> 0 AND
    DISTANCE <> 0
    GROUP BY ORIGIN, DEST
    ORDER BY 5 DESC
    LIMIT 10;
    ''', engine)

display(query_table2)

,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,Total Passengers
0,LAX,"Los Angeles, CA",SFO,"San Francisco, CA",1773482.0
1,SFO,"San Francisco, CA",LAX,"Los Angeles, CA",1749516.0
2,LAX,"Los Angeles, CA",LAS,"Las Vegas, NV",1563481.0
3,LAS,"Las Vegas, NV",LAX,"Los Angeles, CA",1562157.0
4,MCO,"Orlando, FL",ATL,"Atlanta, GA",1498571.0
5,ATL,"Atlanta, GA",MCO,"Orlando, FL",1493988.0
6,TPA,"Tampa, FL",ATL,"Atlanta, GA",1072988.0
7,ATL,"Atlanta, GA",TPA,"Tampa, FL",1062648.0
8,OGG,"Kahului, HI",HNL,"Honolulu, HI",1050416.0
9,HNL,"Honolulu, HI",OGG,"Kahului, HI",1046956.0


,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,Passenger:Distance Ratio,Distance
0,OGG,"Kahului, HI",HNL,"Honolulu, HI",10504.16,100.0
1,HNL,"Honolulu, HI",OGG,"Kahului, HI",10469.56,100.0
2,LAX,"Los Angeles, CA",LAS,"Las Vegas, NV",6624.92,236.0
3,LAS,"Las Vegas, NV",LAX,"Los Angeles, CA",6619.31,236.0
4,LIH,"Lihue, HI",HNL,"Honolulu, HI",6318.85,102.0
5,HNL,"Honolulu, HI",LIH,"Lihue, HI",6317.63,102.0
6,LAX,"Los Angeles, CA",SFO,"San Francisco, CA",5262.56,337.0
7,PDX,"Portland, OR",SEA,"Seattle, WA",5232.66,129.0
8,SFO,"San Francisco, CA",LAX,"Los Angeles, CA",5191.44,337.0
9,SEA,"Seattle, WA",PDX,"Portland, OR",5145.25,129.0


From the results above, we see in the first table clear geographical patterns or regions of where the most popular short-haul flights are from - those being the Southwest (Arizona, California, Nevada), Southeast (Georgia, Florida), and Hawaii. This is in constrast to some of the results above with respect to the airports with the highest amount of short-haul traffic. The second table calculates a Passenger/Distance ratio and orders by highest to find where the most passengers per mile are travelling. Interestingly, two routes in Hawaii are highest and routes in the Pacific Northwest are shown due to their short distance apart. 

For someone involved in development of State or Federal Transportation policy, this could be very useful information in understanding the patterns of movement and where supplemental options for travelling could be implemented.

__Summary:__

Looking at the whole of the United States, about 30% of all commercial passenger traffic is short-haul, or less than 500 miles.

We developed a couple of different methods (Weighted Mean Flight Distance, Short Haul Passengers as a % of Overall) of exploring and manipulating the provided dataset on a city-scale and found that a couple of cities particularly stand out as it pertains to short-haul traffic:
- *Atlanta*, for having the most short-haul passengers in the United States, and 60% more than the next highest location
- *Charlotte*, for having the 9th highest overall passenger traffic, but the 3rd for number of short-haul passengers

We saw that there was a reasonably clear geographic pattern that exist in popular short-haul routes. Short-haul routes in the Southwest (California, Nevada), Southeast (Florida, Georgia), and Hawaii are the most popular generally. When using a Passenger/Distance Ratio, the routes in Hawaii and the Southwest are magnified with the Northwest (Washington, Oregon) replacing those in the Southeast. 

## Data exploration
*(As a team, create a set of queries which draw data from the datasets chosen by multiple team members. The dataset of each team member should be used in conjunction with one or more datasets from more than one team member. The queries should tell you something about your datasets together, which queried alone, would not be obvious or possible to extract from your dataset. What did you learn from the total collection of your chosen datasets? Discuss the steps you took to combine your datasets. Show your results, and the work you did to produce your results.)*
***

***
### Merge 2:  Passenger Traffic dataset with Average Domestic Airline Itinerary Fares dataset
The datasets were merged using the departure airport code or "Origin" in each dataset. Using the 'Weighted Mean Flight Distance' developed above in Dataset \#3, Query \#4 along with the Average Fare of each airport from Dataset \#1, we can create a '$/Mile' ratio highlighting airports which theoretically provide the least or best value.

The following merge and query were then performed:

In [88]:
# Looking at all airports, output average fare/weighted mean ratio and rank from best to worst
best_airport_value = pd.read_sql_query(
    '''
    SELECT ORIGIN, ORIGIN_CITY_NAME as `City Name`, round(AVG_FARE.Fare/WTD_MEAN.WeightedMean,2) as `Avg $/Mile`
    FROM 
    (
    SELECT ORIGIN, ORIGIN_CITY_NAME, (SUM(PASSENGERS*DISTANCE)/SUM(PASSENGERS)) as `WeightedMean`
    FROM air_carrier_statistics
    WHERE PASSENGERS <> 0
    GROUP BY ORIGIN
    ) WTD_MEAN
    INNER JOIN
    (
    SELECT `Airport Code` as `Org`, `Average Fare ($)` as `Fare`
    FROM Average_Fare_Annual_2019
    ) AVG_FARE
    ON WTD_MEAN.ORIGIN = AVG_FARE.ORG
    ORDER BY 3 ASC
    ;
    ''',
    con=engine)
display(best_airport_value)

# Filter the T20 for traffic, output average fare/weighted mean ratio and rank from best to worst
print('Taking the Top 20 Airports for Passenger Traffic and Ranking their Theoretical Value')
best_airport_value_t20 = pd.read_sql_query(
    '''
    SELECT WTD_MEAN.ORIGIN, ORIGIN_CITY_NAME as `City Name`, RANK_TAB.TOT_RANK as `Passenger Rank`, round(AVG_FARE.Fare/WTD_MEAN.WeightedMean,2) as `Avg $/Mile`
    FROM 
    (
    SELECT ORIGIN, ORIGIN_CITY_NAME, ORIGIN_STATE_ABR, (SUM(PASSENGERS*DISTANCE)/SUM(PASSENGERS)) as `WeightedMean`
    FROM air_carrier_statistics
    WHERE PASSENGERS <> 0
    GROUP BY ORIGIN
    ) WTD_MEAN
    INNER JOIN
    (
    SELECT `Airport Code` as `ORG`, `Average Fare ($)` as `Fare`
    FROM Average_Fare_Annual_2019
    ) AVG_FARE
    ON WTD_MEAN.ORIGIN = AVG_FARE.ORG
    INNER JOIN
    (
    SELECT ORIGIN, SUM(Passengers) as `TOT_PASS`, RANK() OVER(ORDER BY SUM(PASSENGERS) DESC) as `TOT_RANK`
    FROM air_carrier_statistics
    GROUP BY ORIGIN    
    ) RANK_TAB
    ON RANK_TAB.ORIGIN = WTD_MEAN.ORIGIN
    WHERE TOT_RANK < 20
    ORDER BY 4
    LIMIT 10
    ;
    ''',
    con=engine)
display(best_airport_value_t20)


,ORIGIN,City Name,Avg $/Mile
0,BFM,"Mobile, AL",0.11
1,AZA,"Phoenix, AZ",0.14
2,PGD,"Punta Gorda, FL",0.14
3,ACY,"Atlantic City, NJ",0.15
4,LBE,"Latrobe, PA",0.15
...,...,...,...
433,CPX,"Culebra, PR",9.87
434,CYF,"Chefornak, AK",10.38
435,ENA,"Kenai, AK",10.41
436,VQS,"Vieques, PR",17.13


Taking the Top 20 Airports for Passenger Traffic and Ranking their Theoretical Value


,ORIGIN,City Name,Passenger Rank,Avg $/Mile
0,LAS,"Las Vegas, NV",6,0.23
1,LAX,"Los Angeles, CA",5,0.25
2,MCO,"Orlando, FL",10,0.25
3,JFK,"New York, NY",18,0.26
4,SEA,"Seattle, WA",8,0.26
5,SFO,"San Francisco, CA",11,0.28
6,BOS,"Boston, MA",13,0.30
7,DEN,"Denver, CO",3,0.31
8,EWR,"Newark, NJ",16,0.33
9,PHX,"Phoenix, AZ",9,0.33


The first table above outlines for the 437 airports we have average fares for, those that provide the best and worst value. From the results, we can see that the top five are composed of airports in reasonably remote areas (Mobile, AL) or those that are low-cost carrier hubs (Atlantic City, Punta Gorda, Latrobe, Pheonix). 
The airports for theoretical worst value are those in Puerto Rico, Alaska, and the U.S. Virgin Islands. 

The second table looks to isolate those airports in the Top 20 for Passenger Traffic and examine the best value. Here, we find Las Vegas the best value. This is somewhat expected given how it is mainly marketed as a vacation destination. 


***
## Discussion
*(Talk about what each individual learned from this project. Are there technologies or techniques you feel would be a better fit now, a particular set of tasks you would do differently now, or ways that you would extend this project in the future?)*

### For Dataset #3 (David):
<u>What I learned:<u>
- I have learned some new SQL techniques such as the *WITH* statement and window functions such as *RANK*
- To look at all available columns and related key legends prior to investigating a dataset - doing so will save time over the course of the project

<u>What I would do differently:<u>
- I would try to more firmly define an end goal or conclusion. My objective(s) for the dataset were slightly vague. 
- I reused a particular query multiple times in both my own dataset analysis as well as in the merge query. I would like to figure out a way within the Python/SQL pipeline this project operated in to reuse a query's code multiple times similar to a function call rather than repeat/copy and paste the code. 

***
## Conclusion
(*Summarize the project, especially the parts you feel were most noteworthy.)*

The project investigated 4 separate datasets that all related to seperate topics (average fare, on-time performance, passenger traffic, Twitter US airline sentiment) within the commercial aviation industry. By investigating each of our datasets separately using SQL queries, we were able to draw insightful conclusions on each specific topic within the commercial aviation industry. 

<u>For the passenger traffic dataset, we found:<u>
- 30% of all passengers are travelling on short-haul flights (less than 500 miles)
- Charlotte and Atlanta particularly stand out as being areas where short-haul traffic is high
- There are clear geographical patterns in popular short-haul routes within the Southeast (GA, FL), Southwest (CA, NV), the state of Hawaii, and Northwest (WA, OR).                                                                                                         
<u>For merge #2 (Average fare dataset and passenger traffic dataset), we found:<u>
- The top 5 and bottom 5 best value airports by average fare dollar per mile
- The best value airport among the top 20 in total traffic (Las Vegas)

Overall, our project allowed us to gain a thorough understanding of major topics that relate to the pre-pandemic commercial aviation industry. 
The datasets used in this project are incredibly rich with many other avenues to explore beyond what we have done here.


***
## References

Bui, Q., Kliff, S. (2021). New York Times - Air Travel Is Already Back to Normal in Some Places. Here’s Where. https://www.nytimes.com/2021/04/01/upshot/flights-rebounding-vacations.html 
 

Creative Commons (2021). Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0) https://creativecommons.org/licenses/by-nc-sa/4.0/  


Kaggle (2015). Twitter US Airline Sentiment. https://www.kaggle.com/crowdflower/twitter-airline-sentiment 


Tippett, R. (2019). Raleigh and Charlotte are among fastest-growing large metros in the United States | Carolina Demography. Retrieved 10 December 2021, from https://www.ncdemography.org/2019/07/24/raleigh-and-charlotte-are-among-fastest-growing-large-metros-in-the-united-states/

US Department of Transportation (2021). Bureau of Transportation Statistics – Domestic Market All Carriers. https://www.transtats.bts.gov/DL_SelectFields.asp?gnoyr_VQ=GED 
 

US Department of Transportation (2021). Bureau of Transportation Statistics – Average Domestic Airline Itinerary Fares. https://www.transtats.bts.gov/AverageFare/ 
 

US Department of Transportation (2021). Bureau of Transportation Statistics – Airline On-Time Performance Data. https://www.transtats.bts.gov/TableInfo.asp?gnoyr_VQ=FGJ&QO_fu146_anzr=b0-gvzr&V0s1_b0yB=D 
 

US Department of Transportation (2021). Bureau of Transportation Statistics – Master Coordinate. https://www.transtats.bts.gov/DL_SelectFields.asp?gnoyr_VQ=FLL . 

